<a href="https://colab.research.google.com/github/sanyabatraaa/MachineLearning/blob/main/dogs_cats_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'dogs-vs-cats:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F84954%2F196452%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240820%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240820T213516Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D013c5f43ec931c87b09a08e1ed6855f9d529f0ad8745e69e5a8a15b916bdc216b018d08231f520494dd3971e248db10ded3aaa941bef2e04c401665d8a1a225ce907fb8a7ef46be12ef75b4acb9493ff1c27621bae158aa5ce7567500839170a0328350b22008863a718a4b10ef326be1bd89f391efd8ed6eef746af5dd429522b95b9ec04d7c7fe832092351d14c15f9f8fc663fa9e36e61413da97486ff3c3cd8b4b403fccfbd773d2c39f41f65f7b075c8f32604b230c9235e5b9faf302e0f6759e8c46511aa4c2141197098d69be0850e6631ebf344791d0ecfe61463758a9b3c2e88513a1e739d2c479f014e67983be17ce111b3a2913131b4411efc0ab'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 1142891694 bytes downloaded
Downloaded and uncompressed: dogs-vs-cats
Data source import complete.


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Streaming output truncated to the last 5000 lines.
/kaggle/input/dogs-vs-cats/dogs_vs_cats/test/dogs/dog.5978.jpg
/kaggle/input/dogs-vs-cats/dogs_vs_cats/test/dogs/dog.12074.jpg
/kaggle/input/dogs-vs-cats/dogs_vs_cats/test/dogs/dog.3521.jpg
/kaggle/input/dogs-vs-cats/dogs_vs_cats/test/dogs/dog.6059.jpg
/kaggle/input/dogs-vs-cats/dogs_vs_cats/test/dogs/dog.7678.jpg
/kaggle/input/dogs-vs-cats/dogs_vs_cats/test/dogs/dog.5940.jpg
/kaggle/input/dogs-vs-cats/dogs_vs_cats/test/dogs/dog.2577.jpg
/kaggle/input/dogs-vs-cats/dogs_vs_cats/test/dogs/dog.7850.jpg
/kaggle/input/dogs-vs-cats/dogs_vs_cats/test/dogs/dog.10112.jpg
/kaggle/input/dogs-vs-cats/dogs_vs_cats/test/dogs/dog.794.jpg
/kaggle/input/dogs-vs-cats/dogs_vs_cats/test/dogs/dog.119.jpg
/kaggle/input/dogs-vs-cats/dogs_vs_cats/test/dogs/dog.10941.jpg
/kaggle/input/dogs-vs-cats/dogs_vs_cats/test/dogs/dog.11521.jpg
/kaggle/input/dogs-vs-cats/dogs_vs_cats/test/dogs/dog.8799.jpg
/kaggle/input/dogs-vs-cats/dogs_vs_cats/test/dogs/dog.6617.jpg
/k

In [4]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [5]:
train_ds = keras.utils.image_dataset_from_directory(
    directory = ('/kaggle/input/dogs-vs-cats/train'),
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (256,256)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = ('/kaggle/input/dogs-vs-cats/test'),
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (256,256)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [6]:
#normalize value bw 0 and 1
def process(image,label):
    image=tf.cast(image/255.,tf.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [7]:
# Create cnn model

model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 254, 254, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 batch_normalization_1 (Bat  (None, 125, 125, 64)      256       
 chNormalization)                                                
                                                        

In [9]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds,epochs=10,validation_data=validation_ds)

Epoch 1/10
625/625 [==============================] - 532s 848ms/step - loss: 1.1424 - accuracy: 0.5943 - val_loss: 0.6139 - val_accuracy: 0.6604
Epoch 2/10
625/625 [==============================] - 523s 836ms/step - loss: 0.5346 - accuracy: 0.7363 - val_loss: 0.6354 - val_accuracy: 0.7056
Epoch 3/10
625/625 [==============================] - 522s 835ms/step - loss: 0.4444 - accuracy: 0.7930 - val_loss: 0.5008 - val_accuracy: 0.7320
Epoch 4/10
625/625 [==============================] - 521s 833ms/step - loss: 0.4027 - accuracy: 0.8195 - val_loss: 0.4369 - val_accuracy: 0.7952
Epoch 5/10
625/625 [==============================] - 524s 839ms/step - loss: 0.3381 - accuracy: 0.8546 - val_loss: 0.5104 - val_accuracy: 0.7258
Epoch 6/10
625/625 [==============================] - 523s 836ms/step - loss: 0.3256 - accuracy: 0.8662 - val_loss: 0.4145 - val_accuracy: 0.8128
Epoch 7/10
625/625 [==============================] - 522s 834ms/step - loss: 0.2149 - accuracy: 0.9125 - val_loss: 0.4541 -

In [1]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()

NameError: name 'history' is not defined

In [ ]:
import cv2
test_img=cv2.imread('data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBwgHBgkIBwgKCgkLDRYPDQwMDRsUFRAWIB0iIiAdHx8kKDQsJCYxJx8fLT0tMTU3Ojo6Iys/RD84QzQ5OjcBCgoKDQwNGg8PGjclHyU3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3N//AABEIAJQAlAMBIgACEQEDEQH/xAAcAAAABwEBAAAAAAAAAAAAAAAAAQIDBAUGBwj/xAA4EAACAQMDAgMGAwgBBQAAAAABAgMABBEFEiExQQYTUQciYXGBkRSh8CMyM0JSscHR4RUWJNLx/8QAFwEAAwEAAAAAAAAAAAAAAAAAAAECA//EAB8RAQEBAAMBAAIDAAAAAAAAAAABEQIhMRIiQQMyQv/aAAwDAQACEQMRAD8A4cKOhRikqQVGKFGBQrBCjowKHShUW3hzQrnXtQW2gUiMcyy44jXnn8q2Hij2eMlo17orb/KjBktiMEgdSpz+VbLwb4eXQ9DhRx+3k/aStjqx7fQcVo4SrEcAEdKWwu/08zlCrEEEEHBB7URrY+0/Qk0bxCZIFK294nmxjsDnDD71j9vuk/HFCp4TiiNHQIoGEGipeMnArV+GfAeqa7CtyxW0tGxsmlH7/wAhTRYyNHVx4q0uLRNZm06GUyiAAM7dyRn/ACKp6CFQozRUAKOio6AKjFEKOgDoxQAo+lC4AqZpESz6rZQsPdkuI1I9QWAqJV94IgEviexL/uRuZG+SjP8AfFI8egLtlVgg7KKg79rgr61Ev7zZdwhs+8Mdf18al2t1EschZAGHc9MUr2qdRTe0jQG17wy81tEWvbD9soHVk/nX7YP0riEkZSAZUj3yOR8BXpeymMTIWxxwR/UP/lUGteAtH1RWiiU26ZZgYgMjP+qaNxwDHei68Cu3aj7JNNlgY2d7LGzAbQ4BVTj4dqY0z2U29tqKSXrF4VbOzOQePWgfUZb2deBG1dl1PVomXT0OYo2GPxB/9f712U26pBtVFUAYAAwAKmLbxRKkcSqkaABVUYAA7VX+I7n8NoGpXIcKI4HIbOMYBplrzd4hu/xut6hcjkS3DsPgu7gfbFVtAnJyetFQQzRUDRUJo80KKhQNKFHQFHihcgClCio6F4Otd7OrV5dTmkCtsWPazDpyRxWSFdP9nqLFoDMqndJPuJx6UsLlci31q4kGtW0ef2QxtH0/X3q3hkIlJdT5bJjNV99pqX10lxJIy7G3bV71OgBkfG7p9qiHeXWJ0MoVFUs2U4GRzj9ZqYs0nkPJGCduCV+Gar0tzuLZLIg3HZyRiq//AL80221GSwFssmxP2o5VgMZPPTIHaqktZ243Vk/nWiOMEMM1Ktz5sewr3qts7y2NpavZZ/DOvB789jV3ZIGjLZAGScmmWoE6FU91sD+o1jfaRKW8GamnmeSm1V3HkkbhmtyGguUxbzJLjqVNYzxxEo8Ka0LsKY0iBAPHf19acGzHno28bNiG4Rz6MCpP34/OmHRkOGUg+lP3CR58yEkoexPK/CmSxxjJx6UAiio8UVMgoUKFAwsUqkilCk1gUoUVHSUNa6H7PJybKaHoAwbqOvyrngrVeBr8292Ynm2xkcg0FznTp3mALwfrSYAZ3IH8Om8AqOMgjp607DMsC4jTJFQieLjR5pLObjDRNwyn0+VVus+E9Oa5uL6zuFtmnhaB1EO87SckA56574zjjNCOeaQ591T2OanWRcBvfAbrkDn8+KrjysTy4yqzQXlsIBZyFlgjbKB+vHrW1tp/xuj3EELAl0IU/OqWXTob+3MNzIw3dJVb3gexp/RbS6tY0tGnUmJv4wTBIzkcD4U5bKufxzlxt1W+BdL1XR9Ta2vrm4uDcSMxWUZWNRnDA+nI4PccVYe03S5tS8K3cNq21zgsFGd2Oa1TSiQNtJAz0B61n/G1/HYaDO0jYLDA4B/uRWlu1hJkeXHjaPO8YPTFNmpt5FJPcySJsfcSfdlRj9gaiyQyxfxI2X5ioam6KjIoqCChQoUAoU5jims0vPFFXxpY5FLRQcjPNMg0oMRSxpqX5AMZOeRVj4aija8G6bY/bg/b51WW/nTuI4+SeOnFdC8GeGysZmvbdWHXzEIYEfHNEiOVyNTYAiJEb0FTzAc5ABqBG+y4O3AA4C5q4gJIGetRURHt7dY2JxgmpC7kk3EZA6c1JEIYZA59RTy2LuOhNEKnIlkvPLEf7NfQtkmpBsJrSdLiNjskOHUnOD2NOaZZGBixq1Miou5sAAdScYrSD7s8NZKxEcjPJrmHtQ1YzobXazwqdmY5NrFvTuMnsD1qd7QfHqaXD+Hsv2m84d0IJX5dq5DquqXdvrDu87TxOAdrElXRucc/r86acVr2cUwZrCUybRkwyACQfLHDfTn4VF3yJ7u44HVT0+1TdRCFxd2/EUg556H9fmDUOWbzlzJ++P5u7D4/H40lG2IPIAHwFJoUVBhQoUKCClCkClUCUoUrNNg1P0nTbjU7wW1uu5up+AoXrQeBdMW+vssxG3sUyv1rr6RJBAltEEVRy20cE1U+F/D66Tp679hk68DFW8RPnBlJB685otyMrfqnoNIR8nYFb0NWUeneTGDIVAHfOakWc4kUJJ7rdiKlNCc4kUOOuTUK01YwrMfcGVHfFW8NvGAMrTNuURAFAFSDPGjYJFVE0HiUcKKgatppv7RoEuGhc/ustS5Jt38MZoog+8Mx+lEvYs6eX/HGhPomqSrLc72c5wTknp+uTVBeSGSK13E7kjKfQMcfka7Z7ZfD8zWb3tqS6pktHjoOOR+dcLbcxz1qqc7SI5nSzZcKUYleR04B4/KotLyxQISdoyQKTtPpSPKKhR7T6UexvSg8pNClbG/pNCgspYi+NLEIP81XDadJBCXeBlXOCfLPFRGmiP7pCjucZ5pNckJsNOa7uo4IuWdsda7H4W8MWui2KT3kMbTYyGA5/Lisf4DjU3Hns0e1eo6ZrbXWptcEon7nQDtTlkjHnduRKuL5pZPRVPSn7XUInYeYMHviqJ5WUE4xil2y4AI6k85NRackbGC8tuNsi5PZxirKK4AXjkN6c1igxJDZPXFPDUSvuhiAKNP5bbz4yR7wyOtIa5iV8SMGHaqbRblJSQzb/hVlJZ28yyOEwVGeMn8qZYlxX0fABGaWLgZzmqm3QMckYPpU9Y9y/KjRYTrccN9pkqTRh1C8ivM/iO2NnqUy+QFiDHaRjB+1elHcxNg9MdK4x7RvDF3Pqxuop4mVjgDBGB9z/ir9hcblc781Ou3FH5imrA+HbwAkyRYHXmiGgT97iEUsi5yqEpye2KVHtLYZsD5VL/6JIDzeQinIdKkiO9L2Ljr7uaWK+jDQQ7jsnLr2bYRn6UKe/wCnsxJ/HKTnnAoUYPqNToWq3ayOt4Sy491mUMPqasGFjKwk/D2as4745Hoaxmn3kltcywMehIy/Ofn/AIqvu53gvjKrBkzkYORzU5qdx0z/AMXyfKgjjRcfyjbWWuddm028eLeU2ngPyKVp2pTPagCMKvoiggiqbxIVdoiUAIOA+c7hU8fVcvOm80rXLbVYVzJGrjgqp6mruORCBtOeO9cbS0Jh82EMrjqC39qsbDXtR0vPmMZEzyrk/wB6pnljqxkJOQenGaOONJpeTzWX8OeJodQOy5IjOeueK2y2qRGO5hO+PI3gelLDlWthGunBJH/hycBs9D8avkKSowUDc67cioBCahp7W8RG9hlCfXtT2jBo9PTzCA6nnBziqkFpFuFhbySxklH7zU9c3S2kZdyFA7mmdPdJLh8f1nJ9a517WPE0xjNtp+VtkJDS+rfD/mmVSPEvjm3DyRBxGVBxIvUHFVej3s+q2u+85U8ozcn48GuUgTTkkK7nuRzWx0G/eysFUOVeP3h96jl0vh22M6W0FpJMscci4IcFQcjuKycd1DIWaC0tVKPn3o+3zHzqbLq5vbKfaV2PzhOny+B5rPRWN3FeL/KjMCwKnnngGlx8O+tXJJb3MYW5tLZnGMsYgCPrioMP4YM6wQx+WMgoyggfKhOyPYThZsbUJBDc8c4qj8PyySFgSQoYHPpRtzT61cyYlIZV2YGMLgUKo7nWlt7h4kLYU+g9KFG8i/E1H5c8hYcSIMHHcVC1i3TcJIQ5B6+gp2RjZ6g0yjMcgz8vhVlayC6y4AVRywIFHc7FyzFdpjzNAY8MMdxRahayMnmbTIPnyv8AxVlLJDEMExj0Kjr86r5mlvG2q3lqQc8dR8qJ7o/WHdIR3TeCrgDgeopjU4yZdgztkPQckU5p0N1AxAkBRT0IqTPCnmQyTqcL8eAc0dbo7zEOCwlgtjMj+6Rz2rYeGvHcsOmPY3FuZnlYorseAMD61VwrFeCWDzAwHJA4P66VFZYLcpHBgBJPr+v9UfRXi2mjeMrqygkRh5s6gorAj71dR+PdIstGMhlc3WzaYtvO751yS8doQHjYnaMk569/9U2tx58YIUNjPanL+yvCNtp3tHvoUmVVRgS3vEYwD/msle6350jpLIxTqoxkLUBmWWB0i2huwPeq4IVnCOMc9DTk+vTt+fFxHcW7xCOBsAcnsTUaa6MK+WhbcRhhUUqIpcqcjvx0oJh5fM6/CjJo24udJjdYXVn2IVJDepPr8P8AYq9t7lZoAzOocZC+9znt/mspa3QkRoZTgMMA9anWVlJB5bBy0e/JA659fl0+1LBpUdpcTedAu5oo3OAOARng1YWsEOnIo35ZhyfkeP18KlxSpBumBVSF2k9eO2fuazerSXA1BUaUqABjHp1BpT8hfx7Rr7T55LqSSJC6sc5oUJJL2JtozjqCO4oVp2nIsLNFuPLhmG6N+CD/AHoaeNguFH8jEDPyoUKznjSqae4kefczZOfpUqxJZwckZPOKFCnfEz1MeR47ZmRiMkrj4UzayvcxyJMxYAgUdClPDo9LzDeLsZvePOT6Go+qsUuFlQ7WcljjpmhQqp6m+I73UrWzKSMHg8VNtABp4I/mBBo6FF8OK2MlLjK8EYqwvlVzGSBmioUX0p4gXLluDiitzwPnRUKf+Sn9ktYkMgBHbNW5meKxYJxtKlSecZHT5UKFRVqae9uGyhkyD14qZYxregfiMsRwDnFChVVEWaRokaqFyAO9ChQpNH//2Q==')
plt.imshow(test_img)

In [ ]:
test_img.shape

In [ ]:
test_img = cv2.resize(test_img, (256,256))

In [ ]:
test_input = test_img.reshape((1,256,256,3))
model.predict(test_input)